In [7]:
import pandas as pd
import re
from typing import Union
from rich.progress import track
import requests
from geopy.geocoders import ArcGIS
import plotly.express as px

In [8]:
df = pd.read_csv('../data/Medical_Examiner_Case_Archive.csv')
print(df.shape)
df.sample(1)

(54215, 27)


,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,...,Commissioner District,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,Residence City,Residence_Zip,OBJECTID
23022,ME2017-01774,04/15/2017 10:06:00 PM,04/15/2017 10:31:00 PM,44.0,Male,Black,False,ACCIDENT,FENTANYL TOXICITY,NaN,...,5.0,241 WEST 107TH STREET,CHICAGO,60628,-87.629726,41.699595,"(41.699595191453646, -87.6297262418586)",Chicago,60628,69250


In [9]:
df2 = df[df['Incident Address'].notna()]
df2.shape

(53488, 27)

In [10]:
df2['Incident Address'].value_counts()

Unknown                        92
UNKNOWN                        90
UNKNOWN LOCATION               11
9700 GROSS POINT ROAD          10
10000 W O'Hare Ave             10
                               ..
1540 Mandel Avenue              1
W. 95th at S. Turner Ave        1
2751 N Lake Shore Drive         1
4416 North Drake Avenue, #2     1
4895 N. Hermitage #1            1
Name: Incident Address, Length: 51331, dtype: int64

In [11]:
'unk' in 'UNKNOWN location'.lower()

True

In [12]:
re.sub(r"apt.*|\#.*|.*nh,", "", "wowzer NH, cool thin # 12".lower()).strip()

'cool thin'

In [13]:
# regex 2 to specify only alphanumeric + '.' for abbreviations and spaces
re.sub(r"[^a-zA-Z0-9.\s]", "", "646 E.51st Street (Apt 1E)").lower().strip()

'646 e.51st street apt 1e'

In [14]:
car_words = ('auto', 'motor')

def deal_with_commas(x: str) -> str:
    if ',' not in x:
        return x.strip().title()
    parts = x.split(',')
    result = ' '.join([z for z in parts if any(y for y in z if y.isnumeric())])
    return result.strip().title()


def remove_apartment_info(x: str) -> str:
    result = re.sub(r"apt.*|\#.*|.*nh,", "", x)
    result2 = re.sub(r"[^a-zA-Z0-9.\s]", "", result)
    return deal_with_commas(result2)


def clean_address(row: pd.Series) -> Union[int, str]:
    cause = row.get('Primary Cause Line A')
    a = row.get('Incident Address')
    # removes if motor vehicle cause
    if pd.notna(cause):
        if any(word in cause.lower() for word in car_words):
            return pd.NA
    # handles 'unknown' and variations
    if pd.isna(a) or 'unk' in a.lower():
        return pd.NA
    return remove_apartment_info(a.lower())


In [15]:
df2['cleaned_address'] = df2.apply(lambda row: clean_address(row), axis=1)
df2['cleaned_address'].value_counts(dropna=False)

<ipython-input-15-e0b38138a7ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cleaned_address'] = df2.apply(lambda row: clean_address(row), axis=1)


NaN                               2368
1301 Lee Street                     18
9615 Knox Ave                       14
345 Dixie Highway                   14
7001 W Cullom Ave                   13
                                  ... 
34 Paxton Ln                         1
7200 S. Union Avenue 1St Floor       1
727 Sandra Drive                     1
4857 West Diversey                   1
7850 W. 183 St                       1
Name: cleaned_address, Length: 47670, dtype: int64

In [16]:
df3 = df2[df2['cleaned_address'].notna()]
df3.shape

(51120, 28)

In [17]:
f"We removed {df.shape[0] - df3.shape[0]} records due to 'null-like' addresses"

"We removed 3095 records due to 'null-like' addresses"

See the benefit of cleaned addresses below:

In [18]:
df3[['Incident Address','cleaned_address']].sample(2)

,Incident Address,cleaned_address
8562,8049 MARYLAND 1N,8049 Maryland 1N
4102,12632 S. AVENUE O,12632 S. Avenue O


In [19]:
df3.columns

Index(['Case Number', 'Date of Incident', 'Date of Death', 'Age', 'Gender',
       'Race', 'Latino', 'Manner of Death', 'Primary Cause',
       'Primary Cause Line A', 'Primary Cause Line B', 'Primary Cause Line C',
       'Secondary Cause', 'Gun Related', 'Opioid Related', 'Cold Related',
       'Heat Related', 'Commissioner District', 'Incident Address',
       'Incident City', 'Incident Zip Code', 'longitude', 'latitude',
       'location', 'Residence City', 'Residence_Zip', 'OBJECTID',
       'cleaned_address'],
      dtype='object')

In [20]:
def city_sub(row) -> tuple[str, bool]:
    res_city_valid = pd.notna(row['Residence City'])
    inc_city_valid = pd.notna(row['Incident City'])
    if pd.notna(row['Incident City']):
        city = row['Incident City'].title().strip()
        subbed = False
    elif pd.isna(row['Incident City']) and pd.notna(row['Residence City']):
        city = row['Residence City'].title().strip()
        subbed = True
    else:
        city = ''
        subbed = False
    return city, subbed

def make_address(row) -> tuple[str, bool]:
    street = row['cleaned_address'].strip()
    city, city_subbed = city_sub(row)
    zip_code = '' if pd.isna(row['Incident Zip Code']) else row['Incident Zip Code'].strip()
    address = f"{street} {city} {zip_code}"
    return address.strip(), city_subbed


def geo_query(row):
    address = make_address(row)
    url = f"https://my-geocoder.herokuapp.com/geocode?address={requests.utils.quote(address)}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError("Invalid response")


def self_geocode(row) -> tuple[float, float]:
    address = make_address(row)
    geocoder = ArcGIS()
    coded = geocoder.geocode(address)
    if coded:
        return {
            "latitude": coded.latitude,
            "longitude": coded.longitude,
            "score": coded.raw.get('score')
        }
    else:
        return None

In [22]:
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()

addresses = df3.apply(lambda row: make_address(row), axis=1)
df3['full_address'] = [a[0] for a in addresses]
df3['city_subbed'] = [a[1] for a in addresses]

<ipython-input-22-e7148a99a660>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['full_address'] = [a[0] for a in addresses]
<ipython-input-22-e7148a99a660>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['city_subbed'] = [a[1] for a in addresses]


,full_address,city_subbed
4,1049 Napleton Avenue,False
7,166 N. Lamon Chicago,True
11,1306 S Kedzie Chicago 60623,False
12,8695 S Archer Ave Chicago 60638,False
13,11901 South Loomis Chicago 60643,False


In [35]:
# see results of city sub
df3[df3.city_subbed == True].loc[:, 'Incident Address':]

,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,Residence City,Residence_Zip,OBJECTID,cleaned_address,full_address,city_subbed
7,166 N. Lamon,NaN,NaN,-87.747912,41.883585,"(41.88358460314286, -87.74791189663985)",Chicago,60644,49301,166 N. Lamon,166 N. Lamon Chicago,True
15,6946 S. DORCHESTER HALLWAY,NaN,60637,NaN,NaN,NaN,Chicago,60649,50138,6946 S. Dorchester Hallway,6946 S. Dorchester Hallway Chicago 60637,True
401,RUSH OAK PARK,NaN,NaN,NaN,NaN,NaN,Oak Park,60302,118956,Rush Oak Park,Rush Oak Park Oak Park,True
555,13617 Kildare Avenue,NaN,60418,-87.725402,41.644220,"(41.64421982142271, -87.7254020225389)",Crestwood,60418,53082,13617 Kildare Avenue,13617 Kildare Avenue Crestwood 60418,True
700,655 W. 65TH STREET- MARJOR LAWRENCE APARTMENTS,NaN,60636,NaN,NaN,NaN,Chicago,60636,49269,655 W. 65Th Street Marjor Lawrence Apartments,655 W. 65Th Street Marjor Lawrence Apartments ...,True
...,...,...,...,...,...,...,...,...,...,...,...,...
53659,4341 WEST ADDISON STREET,NaN,60641,-87.736900,41.946032,"(41.94603227722463, -87.73690006821778)",Chicago,60641,125752,4341 West Addison Street,4341 West Addison Street Chicago 60641,True
53791,2451 N Sacromento,NaN,60647,-87.702007,41.926394,"(41.92639398332069, -87.70200662780348)",Chicago,60647,120370,2451 N Sacromento,2451 N Sacromento Chicago 60647,True
53797,O'Hare,NaN,NaN,NaN,NaN,NaN,Stuart,34997,123728,Ohare,Ohare Stuart,True
54080,FRANSCICAN INDIANA,NaN,NaN,NaN,NaN,NaN,Dolton,60419,124647,Franscican Indiana,Franscican Indiana Dolton,True


In [24]:
df3.city_subbed.value_counts()

False    50635
True       485
Name: city_subbed, dtype: int64

In [ ]:
geolocator = ArcGIS()
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0)
df3['geo_location'] = df3['full_address'].progress_apply(geocode)

In [ ]:
df3['coded_lat'] = df3['geo_location'].apply(lambda x: x.latitude if pd.notna(x) else None)
df3['coded_long'] = df3['geo_location'].apply(lambda x: x.longitude if pd.notna(x) else None)
df3['coded_score'] = df3['geo_location'].apply(lambda x: x.raw.get('score') if pd.notna(x) else None)

df3.drop('geo_location', axis=1, inplace=True)
print(df3.coded_score.describe())

## Calculate geopy distance from original lat/long

In [16]:
# use if need geocoded data
dff = pd.read_csv('../data/version3.csv')

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (20,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
from geopy import distance

def calc_distance(row):
    if pd.isna(row.latitude) or pd.isna(row.longitude) or pd.isna(row.coded_lat) or pd.isna(row.coded_long):
        return None
    d = distance.distance(
        (row.latitude, row.longitude),
        (row.coded_lat, row.coded_long)
    ).km
    return d

In [ ]:
df3['distance'] = df3.apply(lambda row: calc_distance(row), axis=1)

In [18]:
df3.distance.describe().round(2)

count    46164.00
mean        11.86
std        295.27
min          0.00
25%          0.00
50%          0.00
75%          0.02
max      14996.75
Name: distance, dtype: float64

In [57]:
df3.coded_score.describe().round(2)

count    51105.00
mean        98.19
std          3.64
min         70.00
25%         98.53
50%         99.43
75%        100.00
max        100.00
Name: coded_score, dtype: float64

In [ ]:
df3.to_csv('../data/version3.csv', index=False)